# Лабораторная работа №5. Сверточные нейронные сети

В данной лабораторной работе рассматривается решение задачи классификации посредством свёрточных нейронных сетей.
Для построения сетей предлагается использовать [Keras](https://keras.io) на базе [TensorFlow](https://www.tensorflow.org).

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
import sklearn as sk
import sklearn.datasets
import sklearn.model_selection

mpl.rcParams['axes.grid'] = True

print("tensorflow =", tf.__version__, "keras =", tf.keras.__version__)
if tuple(map(int, tf.__version__.split('.'))) < (2, 5, 0):
    tf.autograph.set_verbosity(0)
    import logging
    logging.getLogger("tensorflow").setLevel(logging.ERROR)

## Задание №1

Загрузите датасет MNIST с помощью вызова `tf.keras.datasets.mnist.load_data()`.
Промасштабируйте обучающую и тестовую выборки, чтобы входные значения лежали в отрезке $[0; 1]$.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

???

print(x_train.dtype, x_train.shape, y_train.shape)
assert x_train.dtype == np.float32 and x_test.dtype == np.float32, 'Неправильный тип входных данных'
assert x_train.shape[1:] == (28, 28, 1) and x_test.shape[1:] == (28, 28, 1), 'Неправильная размерность входных данных'
assert (x_train <= 1).all() and (x_test <= 1).all(), 'Неправильный диапазон входных данных'

Изучите приведённую ниже сеть с архитектурой:

1. Свёточный слой с 16 фильтрами размера $3\times3$ и функцией активации relu
1. Слой MaxPooling с размером шага 2
1. Свёточный слой с 32 фильтрами размера $3\times3$ и функцией активации relu
1. Слой MaxPooling с размером шага 2
1. Свёточный слой с 64 фильтрами размера $3\times3$ и функцией активации relu
1. Слой линеаризации
1. Полносвязный слой с количеством нейронов равным количеству выходных классов с функцией активации softmax

В качестве градиетного спуска выступает SGD, размер батча определяется при обучении модели.
Для вычисления функции потерь используется кросс-энтропия по категориям с автоматическим OneHot кодированием.
Метрика точности соответствует $accuracy(\tilde{y}, y) = \frac{1}{N} \sum_{n=0}^{N-1} 1(\tilde{y}_n = y_n)$.
При этом точное название метрики `sparse_categorical_accuracy`, которое можно сократить до `accuracy`, так как метрика выбирается согласно функции потерь.

Запустите нейросеть.
Итоговая точность должна получится около 92% или больше.

In [ ]:
%%time

def create_model1(input_x, input_y):
    inputs = tf.keras.layers.Input(shape=input_x.shape[1:])
    x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D(pool_size=2, padding='valid')(x)
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=2, padding='valid')(x)
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(len(np.unique(input_y)), activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = create_model1(x_train, y_train)
model.fit(x_train, y_train, batch_size=128, epochs=3, validation_data=(x_test, y_test), verbose=True)
score = model.evaluate(x_test, y_test, verbose=0)
print(dict(zip(model.metrics_names, score)))

y_pred = model(x_test)
print('y_test', y_test[:2], 'y_pred', y_pred[:2])

**Вопросы**:

1. Сколько всего параметров у нейронной сети?
1. Сколько всего участвует в обучении векторов из обучающей выборки?
1. Какая точность достигнута?
1. В обучающей выборке в качестве классов используются числа от 0 до 9.
Что получается на выходе у нейронной сети?
Как выходные значения нейронной сети преобразовать в номер класса?

**Ответы**:

1. ...
1. ...

## Задание №2

Попробуйте увеличить точность классификации, например, добавив после каждого свёрточного слоя нормализацию (BatchNormalization) или изменив архитектуру сети.
Объясните целесообразность произведенных изменений.

In [ ]:
%%time

def create_model2(input_x, input_y):
    ???
    return model

model = create_model2(x_train, y_train)
???
score = model.evaluate(x_test, y_test, verbose=0)
print(dict(zip(model.metrics_names, score)))

**Вопросы:**

1. Почему появились необучаемые параметры (non-trainable)?
1. Распишите, какие и сколько параметров входят в BatchNormalization. Для этого воспользуйтесь переменной `layers` и методом `get_weights`.
1. Сколько параметров используется на каждом batch normalization слое?
1. Какая итоговая точность получилась? Стала ли она больше?
1. Как введение новых слоёв повлияло на вермя обучения сети?
1. Попробуйте поставить BatchNormalization слои перед/после MaxPooling. Как меняются характеристики и время обучения?

**Ответы:**

1. ...
1. ...

## Задание №3

Визуализируйте результат воздействия весов последней сети с первого свёрточного слоя на одно из тестовых изображений.

In [ ]:
layer = model.get_layer(index=1)
p = layer(x_train[10:11])

plt.grid(False)
plt.imshow(x_train[10, :, :, 0], cmap='gray')

fig, axes = plt.subplots(4, 4, figsize=(4*4, 4*4))
for row, ax_rows in enumerate(axes):
    for col, ax in enumerate(ax_rows):
        ax.imshow(???)
        ax.grid(False)

**Вопросы:**

1. На что похожи отклики с первого слоя?

**Ответы:**

1. ...

## Задание №4

Подготовьте датасет [cifar10](https://keras.io/api/datasets/) аналогично предыдущему датасету.
Во время первого запуска будет скачиваться архив (около 170 МБ).
Он будет закеширован локально, поэтому последующие загрузки этого датасета будут занимать меньше времени.

Покажите 4 произвольных изображения из обучающей выборки.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

???

print(x_train.shape, x_train.dtype, y_train.shape, y_train.dtype, y_test.shape)

assert x_train.dtype == np.float32 and x_test.dtype == np.float32, 'Неправильный тип входных данных'
assert (x_train <= 1).all() and (x_test <= 1).all(), 'Данные не отмасштабированы'

imshow ???

Постройте сеть с такой архитектурой:
```
convolution2dLayer: 32, 3x3, 'Padding'=1
reluLayer
maxPooling2dLayer: 2, 'Padding'=0

convolution2dLayer: 32, 3x3, 'Padding'=1
reluLayer
maxPooling2dLayer: 2, 'Padding'=0
       
convolution2dLayer: 64, 3, 'Padding'=1
reluLayer
maxPooling2dLayer: 2, 'Padding'=0

fullyConnectedLayer: 128
reluLayer 
fullyConnectedLayer: num_classes
softmaxLayer
```

Изображения цветные, поэтому для первого слоя с конволюцией укажите параметр `data_format='channels_last'`.
При обучении используйте 15 эпох, размер батча 128, оптимизатор градиентного спуска - SGD.
Также можно использовать любую другую архитектуру, объяснив целесообразность её применения.
Протестируйте работу данной сети.
Время обучения сети 5-10 минут.

In [ ]:
%%time

def create_model4(input_x, input_y):
    ???
    return model

???
score = ???
print(dict(zip(model.metrics_names, score)))

**Вопросы:**

1. Какую точность классификации удалось получить?
1. Какие слои не имеют параметров обучения? Почему?
1. Какую размерность имеют входные изображения?

**Ответы:**

1. ...
1. ...

## Задание №5

Попробуйте улучшить точность классификации, добавив нормализацию после свёрточных и полносвязных слоёв.

In [ ]:
%%time

def create_model5(input_x, input_y):
    ???
    return model

???
print(dict(zip(model.metrics_names, score)))

**Вопросы:**

1. Удалось ли увеличить точность классификации введением новых слоёв?
1. Как изменилось время обучения сети? (Обратите внимание на `Wall time`, `total`, а также на время обучения каждой эпохи)

**Ответы:**

1. ...
1. ...

## Задание №6

Попробуйте добавить слой dropout `tf.keras.layers.Dropout(<probability>)` после полносвязного слоя, подобрав оптимальную на Ваш взгляд вероятность удаления нейронов.
Поэкспериментируйте с добавлением слоёв BatchNormalization и без них.

In [ ]:
%%time

def create_model6(input_x, input_y):
    ???
    return model

???
print(dict(zip(???)))

**Вопросы:**

1. Удалось ли увеличить точность классификации?
1. Что влияет больше на увеличение точности: слой Dropout или BatchNormalization?
1. Как изменилось время обучения?

**Ответы:**

1. ...
1. ...